In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl  
import missingno as msno
import warnings

import plotly.express as ex
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.offline as pyo
pyo.init_notebook_mode()

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score as f1
from sklearn.metrics import confusion_matrix

import scikitplot as skplt
from imblearn.over_sampling import SMOTE

warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid")
%matplotlib inline

mpl.rc('font', family='Malgun Gothic')  # 한글 폰트 설정
                                        # 윈도우 폰트 위치 - C:\Windows\Fonts
plt.figure(figsize=(10,6))              # 그래프 사이즈 설정
sns.set(font='Malgun Gothic', rc={'axes.unicode_minus':False}, style='darkgrid') # 마이너스 처리


In [ ]:
bank_churner_df = pd.read_csv("./data/bank_churner.csv")
bank_churner_df = bank_churner_df.drop('cstno', axis=1)

exist_customer_df = bank_churner_df.query('is_churned == 0')
churn_customer_df = bank_churner_df.query('is_churned == 1')

In [ ]:
# 연속형 데이터 그래프 구현 함수
def cont_feature_graphs(column):
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    sns.boxplot(data=bank_churner_df, x=column, y='is_churned', orient='h', ax=axs[0])
    sns.kdeplot(data=bank_churner_df, x=column, hue='is_churned', common_norm=False, ax=axs[1]) # KDE(커널밀도추정())
    axs[0].set_ylabel('')
    axs[1].set_ylabel('')
    plt.show()
    
# 범주형 데이터 그래프 구현 함수
def cat_feature_graphs(column):
    counts = bank_churner_df[column].value_counts() # calculate the counts by education level
    churn_rates = churn_customer_df[column].value_counts() / bank_churner_df[column].value_counts() # attrition rate by education level
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    sns.barplot(x=churn_rates.sort_values().index, y=churn_rates.sort_values(), ax=axs[0])
    sns.barplot(x=counts.sort_values().index, y=counts.sort_values(), ax=axs[1])
    axs[0].set_ylabel('')
    axs[0].set_title('Churn Rates')
    axs[1].set_ylabel('')
    axs[1].set_title('Counts')
    plt.show()

In [ ]:
cont_feature_graphs('age')

In [ ]:
def print_continuous_graphs(column):
    
    fig = make_subplots(rows=1, cols=2)
    fig.add_trace(
        go.Box(df, x=column, facet_col="is_churned", points="all")
    )
    
    fig.add_trace(
    go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
    row=1, col=2
    )
    fig.show()
    

print_continuous_graphs('age')

In [ ]:
import plotly.express as px

fig = px.box(bank_churner_df, y='age')
#fig = px.box(bank_churner_df['age'], facet_col="is_churned")

fig.show()

In [ ]:
import plotly.graph_objects as go

def print_continuous_graphs(column, column_desc):

    fig = make_subplots(rows=3, 
                        cols=1, 
                        subplot_titles=("전체 건수 분포", "유지/이탈별 사분위", "유지/이탈별 분포", "유지/이탈별 누적합 분포"), 
                      # shared_xaxes=True,
                        horizontal_spacing=0.1,
                        vertical_spacing=0.1
                    )

    # 전체 데이터 분포
    fig.add_trace(go.Histogram(x=bank_churner_df[column]), 
                  row=1, col=1
                )

    # Box Plot - 고객 유지, 이탈 구분
    fig.add_trace(go.Box(x=bank_churner_df[bank_churner_df['is_churned']==0][column], 
                         name='유지'), 
                  row=2, col=1
                )
    fig.add_trace(go.Box(x=bank_churner_df[bank_churner_df['is_churned']!=0][column], 
                         name='이탈'), 
                  row=2, col=1
                )

    # 고객 유지, 이탈별 분포
    fig.add_trace(go.Histogram(x=bank_churner_df[bank_churner_df['is_churned']==0][column]),
                  row=3, col=1
                )
    fig.add_trace(go.Histogram(x=bank_churner_df[bank_churner_df['is_churned']!=0][column]), 
                  row=3, col=1
                )

    #fig.update_layout(width=2000,height=1000)
    fig.update_layout(width=800, 
                    height=1000, 
                    showlegend=False,
                    title_text=f"{column_desc}에 따른 분석 그래프"
                    )

    fig.show()

print_continuous_graphs('age', '나이')

# x = bank_churner_df[bank_churner_df['is_churned']!=0]['age']
# hist_data = [x]
# group_labels = ['distplot'] # name of the dataset

# fig = ff.create_distplot(hist_data, group_labels, show_rug=False)
# fig.show()


In [ ]:
# I largely keep the codes and comments the same as the original answer, with the modification highlighted under '#######'
import plotly.express as px
import plotly.subplots as sp

my_df = px.data.medals_long()

#fig = px.box(bank_churner_df, x='age', color="is_churned")
    
# Create figures in Express
#figure1 = px.bar(my_df, x = "nation", y = "count", color = "medal")
figure1 = px.box(bank_churner_df, x='age', color="is_churned")
figure2 = px.line(my_df, x = "nation", y = "count", color = "medal")

# For as many traces that exist per Express figure, get the traces from each plot and store them in an array.
# This is essentially breaking down the Express fig into its traces
figure1_traces = []
figure2_traces = []

for trace in range(len(figure1["data"])):
    figure1_traces.append(figure1["data"][trace])
    
for trace in range(len(figure2["data"])):
    ############ The major modification. Manually set 'showlegend' attribute to False. ############
    figure2["data"][trace]['showlegend'] = False             
    figure2_traces.append(figure2["data"][trace])

   
# Create a 1x2 subplot
this_figure = sp.make_subplots(rows = 1, cols = 2, subplot_titles = ['Box', 'Line'])
this_figure.update_layout(height = 500, width = 1200, title_text = "Medals count by country", title_font_size = 25)

# Get the Express fig broken down as traces and add the traces to the proper plot within the subplot
  
for traces in figure1_traces:
    this_figure.append_trace(traces, row = 1, col = 1)
    
for traces in figure2_traces:
    this_figure.append_trace(traces, row = 1, col = 2)
    
this_figure.show()


In [ ]:
import plotly.express as px

# 데이터 불러오기
df = px.data.tips()

fig = px.box(bank_churner_df, x='age', color="is_churned")

fig.show()

In [ ]:
import plotly.figure_factory as ff
import numpy as np
np.random.seed(1)

x = bank_churner_df[bank_churner_df['is_churned']!=0]['age']
hist_data = [x]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, show_rug=False)
fig.show()

#fig.add_trace(go.Histogram(x=bank_churner_df[bank_churner_df['is_churned']!=0]['age'], histfunc='sum'), row=2, col=2)

In [ ]:
help(make_subplots)

In [ ]:
import plotly.express as px

fig = px.box(bank_churner_df, x="sex", y="age")

fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2)

fig.add_trace(go.Scatter(x=[1, 2, 3], y=[4, 5, 6]),
              row=1, col=1)

fig.show()
#fig.add_trace(px.box(bank_churner_df, x="age", color="is_churned"), row=1, col=1)

#fig = px.box(bank_churner_df, x="age", color="is_churned")

fig.show()

In [ ]:
# imports
import plotly.express as px
import plotly.graph_objects as go

# data
df = px.data.tips()

# plotly setup
fig=go.Figure()

# a plotly trace for each subcategory
for i, smokes in enumerate(df['smoker'].unique()):
    df_plot=df[df['smoker']==smokes]

    fig.add_trace(go.Box(x=df_plot['time'], y=df_plot['total_bill'],
                         notched=True,
                         line=dict(color='black'),
                         #line=dict(color=colors[i]),
                         fillcolor='yellow',
                         #fillcolor=colors[i+4],
                         name='smoker=' + smokes))

# figure layout adjustments
fig.update_layout(boxmode='group', xaxis_tickangle=0)
fig.show()

In [ ]:
import plotly.express as px

fig = px.box(bank_churner_df, x="age", facet_col="is_churned")

fig.show()
facet_col="sex"

In [ ]:
bank_churner_df.info()

In [ ]:
from plotly.subplots import make_subplots
import plotly.express as px

def cont_feature_graphs(column):
    
    fig = make_subplots(rows=2, cols=1)

    tr1=go.Box(x=bank_churner_df['age'], facet_col='is_churned', name='나이 박스 플롯', boxmean=True)
    #tr1=px.box(bank_churner_df, x=column, color='is_churned')
    tr2=go.Histogram(x=bank_churner_df[column], name='나이 히스토그램')

    fig.add_trace(tr1,row=1,col=1)
    fig.add_trace(tr2,row=2,col=1)

    fig.update_layout(height=700, width=1200, title_text="나이별 분포")
    fig.show()
    
    
cont_feature_graphs('age')

In [ ]:
# 연속형 데이터 그래프 구현 함수
def cont_feature_graphs(column, xlabel):
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    sns.boxplot(data=bank_churner_df, x=column, y='is_churned', orient='h', ax=axs[0])
    sns.kdeplot(data=bank_churner_df, x=column, hue='is_churned', common_norm=False, ax=axs[1]) # KDE(커널밀도추정())
    axs[0].set_ylabel('')
    axs[0].set_xlabel(xlabel)
    axs[1].set_ylabel('')
    axs[0].set_xlabel(xlabel)
    plt.show()

In [ ]:
cont_feature_graphs('age','나이')